In [8]:
import pandas as pd
import numpy as np
#一 数据收集 start 
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [9]:
#二 数据预处理 start 
print(train['Attrition'].value_counts())
train['Attrition']=train['Attrition'].map(lambda x:1 if x=='Yes' else 0)

No     988
Yes    188
Name: Attrition, dtype: int64


In [10]:
#train.isna().sum()#查看是否有空值 Step1，对数据进行探索 
#Step2，去掉无用特征，处理分类特征
train= train.drop(['EmployeeNumber', 'StandardHours'], axis=1)
test=test.drop(['EmployeeNumber', 'StandardHours'], axis=1)

In [11]:
#分类特征进行特征值编码：
from sklearn import preprocessing 

attr=['Age','BusinessTravel','Department','EducationField','Gender','JobRole','MaritalStatus','Over18','OverTime']
lbe_list=[]
for fea in attr:
    lbe=preprocessing.LabelEncoder()
    train[fea]=lbe.fit_transform(train[fea])
    test[fea]=lbe.transform(test[fea])

In [12]:
from tpot import TPOTClassifier

In [20]:
#Step4，模型训练，得出预测结果

from sklearn.model_selection import  train_test_split
from sklearn.metrics import accuracy_score
X_train,X_valid,Y_train,Y_valid=train_test_split(train.drop('Attrition',axis=1),train['Attrition'],test_size=0.2,random_state=42)
tpot=TPOTClassifier(generations=5,population_size=20,verbosity=2)
tpot.fit(X_train,Y_train)
print(tpot.score(X_valid,Y_valid))


Generation 1 - Current best internal CV score: 0.8723404255319149
Generation 2 - Current best internal CV score: 0.8723404255319149
Generation 3 - Current best internal CV score: 0.8723404255319149
Generation 4 - Current best internal CV score: 0.8723404255319149
Generation 5 - Current best internal CV score: 0.874468085106383

Best pipeline: GradientBoostingClassifier(StandardScaler(input_matrix), learning_rate=0.1, max_depth=10, max_features=0.6000000000000001, min_samples_leaf=15, min_samples_split=4, n_estimators=100, subsample=0.2)
0.8177966101694916


In [22]:
predict=tpot.predict_proba(test)[:, 1]
test['Attrition']=predict
test[['user_id','Attrition']].to_csv('Submission.csv')

In [21]:
tpot.export('tpot_pipeline.py')


In [7]:
TPOTClassifier?

Init signature: TPOTClassifier(generations=100, population_size=100, offspring_size=None, mutation_rate=0.9, crossover_rate=0.1, scoring=None, cv=5, subsample=1.0, n_jobs=1, max_time_mins=None, max_eval_time_mins=5, random_state=None, config_dict=None, template=None, warm_start=False, memory=None, use_dask=False, periodic_checkpoint_folder=None, early_stop=None, verbosity=0, disable_update_check=False)
Docstring:      TPOT estimator for classification problems.
Init docstring:
Set up the genetic programming algorithm for pipeline optimization.

Parameters
----------
generations: int or None, optional (default: 100)
    Number of iterations to the run pipeline optimization process.
    It must be a positive number or None. If None, the parameter
    max_time_mins must be defined as the runtime limit.
    Generally, TPOT will work better when you give it more generations (and
    therefore time) to optimize the pipeline. TPOT will evaluate
    POPULATION_SIZE + GENERATIONS x OFFSPRING_SI